# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [10]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(
        self,
        min_samples_split=2,
        max_depth=None,
        sufficient_share=1.0,
        criterion='gini',
        max_features=None
    ):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum(l_c**2 / l_s + r_c**2 / r_s, axis=1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = (l_s.astype('float'))
        r_s = (r_s.astype('float'))
        return -(np.sum(
            l_c * np.log(
                (l_c + 1e-4) / l_s
            ) + r_c * np.log((r_c + 1e-4) / r_s), axis=1
        )).reshape(-1, 1) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.max(
            l_c / (l_s + r_s), axis=1
        ) - np.max(r_c / (l_s + r_s), axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #получаем x и y, отсортированные по возрастанию x 
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # я удалил следующую строчку
        #и заменил везде class_number на self.num)_class, 
        # чтобы работало на данных с боллее, чем двумя классами
        #class_number = np.unique(y).shape[0]
        #места, где может быть разбиение,
        #не нарушающее условия на минимальное число элементов в листе
        splitted_sorted_y = sorted_y[
            self.min_samples_split:-self.min_samples_split
        ]
        #места, где меняются классы
        r_border_ids = np.where(
            splitted_sorted_y[:-1] != splitted_sorted_y[1:]
        )[0] + (self.min_samples_split + 1)
        #если не меняются нигде
        if len(r_border_ids) == 0:
            return float('+inf'), None

        #число идущих подряд одинаковых индексов (кроме последнего)
        eq_el_count = r_border_ids - np.append(
            [self.min_samples_split], r_border_ids[:-1]
        )
        
        # [i,j] = 1, если i отрезок - имеет номер j
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[
            np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]
        ] = 1
        
        # [i,j] = n, если i отрезок - имеет n элементов номера j
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        #добавляем, что отрезали в splitted_sorted_y
        class_increments[0] = class_increments[0] + np.bincount(
            sorted_y[:self.min_samples_split], minlength=self.num_class
        )
        
        #определяем, сколько элементов класса останется
        #в правой и левой ветви при разьиении по каждой границе
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(
            y, minlength=self.num_class
        ) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        #ищем лучшее impurity 
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        #номер элемента, с которого начинается правый класс
        left_el_id = l_sizes[idx][0]
        #возвращаем impurity и усредненный threshold
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):

        #достигнута максимальная глубина
        if depth == self.max_depth:
            #create leaf
            self.__create_leaf(y, node_id, depth, 'maxDepth')
            return

        #если размер меньше min_samples_split
        if x.shape[0] <= self.min_samples_split:
            #create leaf
            self.__create_leaf(y, node_id, depth, 'lessMinSplit')
            return

        #если присутствует только один класс
        if np.unique(y).shape[0] == 1:
            self.__create_leaf(y, node_id, depth, 'onlyOneClass')
            return

        #если доля какого-то класса > sufficient_share
        if np.max(1.0*np.bincount(y) / y.shape[0]) >= self.sufficient_share:
            self.__create_leaf(y, node_id, depth, 'suffShare')
            return

        #далее выбираем по какому признаку провести разделение

        #списки [признак х (impurity_measure, threshold)]
        imp = np.zeros(x.shape[1])
        threshold = np.zeros(x.shape[1])
        features = self.get_feature_ids(x.shape[1])

        for i in features:
            #поиск порогового значения для i-го признака
            imp_i, thr = self.__find_threshold(x[:, i], y)
            threshold[i] = thr
            imp[i] = imp_i

        #нам нужено найти разбиение с минимальным значением
        feature_id = argmin(imp)

        #если не получилось разделить - создаем лист
        if np.isnan(threshold[feature_id]):
            self.__create_leaf(y, node_id, depth, 'thr-nan')
            return

        #делим
        x_left, x_right, y_left, y_right = self.__div_samples(
            x, y, feature_id, threshold[feature_id]
        )
        if y_right.shape[0] == 0 or y_left.shape[0] == 0:
            self.__create_leaf(y, node_id, depth, 'oneIsEmpty')
            return

        self.tree[node_id] = (
            self.NON_LEAF_TYPE, feature_id, threshold[feature_id], 'child'
        )
        self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)

    def __create_leaf(self, y, node_id, depth, crit=None):
        counts = np.bincount(y, minlength=self.num_class)
        probs = counts / np.sum(counts).astype('float')
        self.tree[node_id] = (self.LEAF_TYPE, np.argmax(counts), probs, crit)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold, _ = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [73]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [74]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [75]:
%time clf.fit(X_train, y_train)

CPU times: user 1.92 ms, sys: 1.08 ms, total: 3 ms
Wall time: 1.77 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [76]:
%time my_clf.fit(X_train, y_train)

CPU times: user 16.6 ms, sys: 2.77 ms, total: 19.4 ms
Wall time: 17.5 ms


## Проверка качества работы на wine

In [77]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

In [78]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [11]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb

df = pd.read_csv('Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'],
               prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

## Проверка скорости работы на Speed Dating Data 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)


In [108]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)

CPU times: user 81 ms, sys: 2.52 ms, total: 83.5 ms
Wall time: 83.3 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [109]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)

CPU times: user 76.6 ms, sys: 2.31 ms, total: 78.9 ms
Wall time: 80.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Проверка качества работы на Speed Dating Data

In [116]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5475681789619202

In [118]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5664030853843609

## Задание 3

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

In [13]:
pipeline = Pipeline([('clf', MyDecisionTreeClassifier())])
X = pipeline.fit(X_train, y_train)
print(f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro'))

0.551441114716549


## Задание 4

In [18]:
model1 = DecisionTreeClassifier()
model1.fit(X_train, y_train)
model2 = MyDecisionTreeClassifier()
model2.fit(X_train, y_train)
features_names = df_pair.columns[1:]
print('DecisionTreeClassifier')
print(pd.Series(index=features_names, data=model1.feature_importances_)\
.sort_values(ascending=False).head(10))
print('MyDecisionTreeClassifier')
print(pd.Series(index=features_names, data=model2.feature_importances_)\
.sort_values(ascending=False).head(10))

DecisionTreeClassifier
int_corr      0.090703
shar1_1_f     0.033382
intel1_1      0.028693
fun1_1        0.028105
intel2_1_f    0.026678
fun1_1_f      0.025977
income        0.024263
shar2_1_f     0.023790
exphappy_f    0.023547
shar2_1       0.023544
dtype: float64
MyDecisionTreeClassifier
int_corr      0.069284
age           0.017382
samerace      0.015662
income_f      0.009290
field_cd      0.008234
age_f         0.007119
mn_sat        0.006466
field_cd_f    0.006459
tuition_f     0.006345
amb1_1_f      0.005703
dtype: float64


## Задание 5

In [21]:
param = {"max_depth": range(1, 10),
              "min_samples_split": range(2, 15),
              "criterion": ["gini", "entropy"]}

gscv = RandomizedSearchCV(RandomForestClassifier(n_estimators=10), param, n_iter=10)
gscv.fit(X_train, y_train)
print(gscv.best_params_)

{'min_samples_split': 8, 'max_depth': 9, 'criterion': 'entropy'}
